In [310]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
import datetime

url = "https://api.hypixel.net/skyblock/auctions"

r = requests.get(url).json()
total_pages = int(r['totalPages'])
total_pages
df3 = pd.DataFrame()
def fetch_auction_house(page_number:int) -> pd.DataFrame:
        response = requests.get(f'{url}?page={page_number}')
        auctions_data = response.json()['auctions']
        df = pd.DataFrame(auctions_data)
        df.drop(columns=['auctioneer','profile_id','coop', 'extra','item_lore','tier','item_bytes','claimed_bidders','item_uuid','bids'],inplace=True)
        return df

def clean_dataframe(df:pd.DataFrame) -> pd.DataFrame:
        df['start'] = pd.to_datetime(df['start'], unit='ms')
        df['end'] = pd.to_datetime(df['end'], unit='ms')
        df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
        return df



for i in range(total_pages):
   df3 = pd.concat( [df3, fetch_auction_house(i)])

df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,9a7cc097407948e397ba4e3e5a84dadd,2024-07-15 04:00:32.101,2024-07-17 04:00:32.101,Mathematical Hoe Blueprint,misc,1900000,False,0,2024-07-15 04:00:32.101,True
1,a8b20543c98244cf8b75f30faabe274e,2024-07-15 04:00:32.078,2024-07-17 04:00:32.078,[Lvl 1] Chicken,misc,2000000,False,0,2024-07-15 04:00:32.078,True
2,793d481fe0bc43a4bf6bfacb8dfa8085,2024-07-15 04:00:31.369,2024-07-17 04:00:31.369,Fish Affinity Talisman,accessories,125000,False,0,2024-07-15 04:00:31.369,True
3,5f7704d99ff34fbba59c589a3956b6db,2024-07-15 04:00:30.710,2024-07-17 04:00:30.710,Crimson Chestplate,armor,7900000,False,0,2024-07-15 04:00:30.710,True
4,0331fb6549344924b7f84982904fa41a,2024-07-15 04:00:30.645,2024-07-17 04:00:30.645,Farmer Boots,armor,640000,False,0,2024-07-15 04:00:30.645,True
...,...,...,...,...,...,...,...,...,...,...
217,6017779e31bb4784870ba1d7c95fdf8b,2024-07-01 02:28:52.875,2024-07-15 04:28:52.875,[Lvl 100] Flying Fish,misc,1000000000,False,0,2024-07-01 02:28:52.875,True
218,66aa40bd12134e04b2a37f3cfed56dd4,2024-07-01 02:27:24.738,2024-07-15 04:27:24.738,Matriarch's Perfume,consumables,1900000000,False,0,2024-07-01 02:27:24.738,True
219,7342798dcac14e91899cbce9e12cd1c9,2024-07-01 02:09:42.162,2024-07-15 04:09:42.162,Ancient Frozen Blaze Leggings ✪✪✪✪✪➋,armor,90000000,False,0,2024-07-01 02:09:42.162,True
220,6a42574449694cc28dde0831c212ab7a,2024-07-01 02:07:23.317,2024-07-15 04:07:23.317,Giant Reaper Mask ✪✪✪✪✪,armor,46000000,False,0,2024-07-01 02:07:23.317,True


In [378]:
# Using threading improved avg runtime from 25 - 30seconds to 3 - 4 (Thanks ChatGPT for the help)

url = "https://api.hypixel.net/skyblock/auctions"
r = requests.get(url).json()
total_pages = int(r['totalPages'])


# For loops through the total_number of pages on the market place, requesting JSON for each page
# Each page contains a max of 1000 Items
def fetch_auction_house(page_number: int) -> pd.DataFrame:
    response = requests.get(f'{url}?page={page_number}')
    auctions_data = response.json()['auctions']
    df = pd.DataFrame(auctions_data)
    return df
# Clean DataFrame
# Remove unnescessary code
# Change all timestamps from UNIX to yyyy/mm/dd hour/min/sec/ms format
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['auctioneer', 'profile_id', 'coop', 'extra', 'item_lore', 'item_bytes', 'claimed_bidders', 'item_uuid', 'bids'], inplace=True)
    df['start'] = pd.to_datetime(df['start'], unit='ms')
    df['end'] = pd.to_datetime(df['end'], unit='ms')
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
    desired_order = ['item_name','tier','uuid','starting_bid','highest_bid_amount','category', 'start','end','last_updated', 'claimed','bin']
    df = df[desired_order]
    return df


#ChatGPT helped me with this one, ill need someone to explain this to me
# Use ThreadPoolExecutor to fetch pages concurrently
def fetch_all_pages_concurrently(total_pages: int) -> pd.DataFrame:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(fetch_auction_house, i) for i in range(total_pages)]
        result = pd.concat([future.result() for future in futures], ignore_index=True)
    return result

df3 = fetch_all_pages_concurrently(total_pages)
df = clean_dataframe(df3)

df

,item_name,tier,uuid,starting_bid,highest_bid_amount,category,start,end,last_updated,claimed,bin
0,Diver's Shirt,LEGENDARY,5629143f2aef451ca744281a1a95f302,3800000,0,armor,2024-07-16 05:16:32.051,2024-07-19 21:16:32.051,2024-07-16 05:16:32.051,False,True
1,Day Crystal,RARE,8855d5991b5c4204b03e1a9ca2cc35eb,940000,0,accessories,2024-07-16 05:16:32.039,2024-07-16 11:16:32.039,2024-07-16 05:16:32.039,False,True
2,[Lvl 1] Ghoul,EPIC,768ec8545c534f75a3e786987cbedb97,65000,0,misc,2024-07-16 05:16:31.730,2024-07-16 11:16:31.730,2024-07-16 05:16:31.730,False,True
3,Gemstone Chamber,EPIC,a8e597677345443ca2a168e6e334c9bc,6600000,0,misc,2024-07-16 05:16:31.527,2024-07-18 05:16:31.527,2024-07-16 05:16:31.527,False,True
4,Ornate Zombie Sword,EPIC,711c4383430142628b40085f985a713c,899999,0,weapon,2024-07-16 05:16:31.288,2024-07-17 05:16:31.288,2024-07-16 05:16:31.288,False,True
...,...,...,...,...,...,...,...,...,...,...,...
58472,[Lvl 80] Ghoul,LEGENDARY,79f832fbf8bd405a9074cf9065218f28,24999999,0,misc,2024-07-02 03:43:39.046,2024-07-16 05:43:39.046,2024-07-02 03:43:39.046,False,True
58473,Bustling Farm Armor Chestplate,RARE,a87b187f53b04106b2ec3312cdf17d5d,50000000,0,armor,2024-07-02 03:34:08.097,2024-07-16 05:34:08.097,2024-07-02 03:34:08.097,False,True
58474,Anubis Golden Dragon Skin,MYTHIC,117108e5f7694fd699fcf4face146e53,400000000,0,misc,2024-07-02 03:22:13.780,2024-07-16 05:22:13.780,2024-07-02 03:22:13.780,False,True
58475,Pest Repellent MAX,RARE,5e438faac4e748f9a0fcb7339770e270,459000000,0,misc,2024-07-02 03:17:31.072,2024-07-16 05:17:31.072,2024-07-02 03:17:31.072,False,True


In [387]:
def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return "Item not with specified Parameters"
    return filtered_df

find_item('Portal Dye', 81000000)

'Item not with specified Parameters'

In [393]:
# Search Dictionary 
# USAGE
# Key = item_name
# Value = Maximum value of item
# search_dict = {
#   'Key': Value,
#   'Key': Value,
# }


search_dict = {
    'Portal Dye':100000000,
    'Molten Cloak': 900000,
    'Molten Belt': 900000		
    }

def find_item_under_price(item_name:str, specified_price:int) -> pd.DataFrame:
    # Filter by name
    filtered_df = df[df['item_name'] == item_name]
    # Sort by the values in starting_bid
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    # Query works like an if which allows us to search for rows where bin == True and claimed == False and starting_bid is less than specified_price
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    # Set uuid in filtered_df to be command version /viewauction + uuid
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return pd.DataFrame(columns=df.columns)
    return filtered_df

def find_item_lbin(item_name:str) -> pd.DataFrame:
    # Filter by name
    filtered_df =  df[df['item_name'] == item_name]
    # Sort by the values in starting_bid
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    # Query works like an if which allows us to search for rows where bin == True and claimed == False
    filtered_df = filtered_df.query(f'bin == True and claimed == False')
    # Because the DF is sorted by starting_bid, we can return the first row and that will be the lbin
    filtered_df = filtered_df.head(1)
    # Set uuid in filtered_df to be command version /viewauction + uuid
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return pd.DataFrame(columns=df.columns)
    return filtered_df

# Initialize df4 as a Dataframe for later use
df4 = pd.DataFrame(columns=df.columns)

# Loop through Dictionary
# This is a helper function that works with find_item_under_price
# NOTES: This function returns an appended Dataframe
# Uses search_dict above as the search criteria
def find_item_under_price_helper() -> pd.DataFrame:
    global df4
    for item_name, specified_price in search_dict.items():
        # Call find_item_under_price and set result equal to it
        result = find_item_under_price(item_name, specified_price)
        # Concat result to current working dataframe
        df4 = pd.concat([df4, result], ignore_index=True)
    return df4

# Loop through Dictionary
# This is a helper function that works with find_item_lbin
# NOTES: This function returns an appended Dataframe
# Uses search_dict above as the search criteria
def find_item_lbin_helper() -> pd.DataFrame:
    global df4
    for item_name in search_dict.keys():
        # Call find_item_under_lbin and set result equal to it
        result = find_item_lbin(item_name)
        # Concat result to current working dataframe
        df4 = pd.concat([df4,result], ignore_index=True)
    return df4

#find_item_under_price_df = find_item_under_price_helper()
#find_item_under_price_df

find_item_lbin_df = find_item_lbin_helper()
find_item_lbin_df

C:\Users\jdangelo\AppData\Local\Temp\ipykernel_24164\3254147510.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df4 = pd.concat([df4,result], ignore_index=True)


,item_name,tier,uuid,starting_bid,highest_bid_amount,category,start,end,last_updated,claimed,bin
0,Portal Dye,MYTHIC,/viewauction 47d9b71a0e40453997df27628dc656f0,89900000,0,misc,2024-07-16 05:16:29.156,2024-07-16 11:16:29.156,2024-07-16 05:16:29.156,False,True
1,Molten Cloak,EPIC,/viewauction 34e93d035659497a831c0c2bc1a23367,1000000,0,misc,2024-07-16 05:09:55.175,2024-07-16 11:09:55.175,2024-07-16 05:09:55.175,False,True
2,Molten Belt,EPIC,/viewauction 13b10e081ab448cc91b32cff4e6e18dd,890000,0,misc,2024-07-16 04:10:01.807,2024-07-18 04:10:01.807,2024-07-16 04:10:01.807,False,True


In [319]:
class Shareholder:
    def __init__(self, PersonName:str, InitialItemsInvested:dict):
        self.PersonName = PersonName
        self.InitialItemsInvested = InitialItemsInvested



In [337]:
person1 = Shareholder("Banyan",
                      { 'Portal Dye':78000000,
                        'Molten Cloak': 900000,
                        'Molten Belt': 900000	})

In [344]:
response = requests.get('https://sky.coflnet.com/api/item/price/DYE_LUCKY/history/full').json()
df = pd.DataFrame(response)
CurrentPrice = df.tail(1)
df

,min,max,avg,volume,time
0,49.0,300000000.0,39367900.0,9339,2024-03-16T00:00:00
1,36.0,140000000.0,30757500.0,3123,2024-03-17T00:00:00
2,2690000.0,533333000.0,29860000.0,623,2024-03-21T00:00:00
3,500.0,34000000.0,26123800.0,595,2024-03-22T00:00:00
4,24.0,25000000.0,23816400.0,360,2024-03-25T00:00:00
...,...,...,...,...,...
102,60000000.0,71500000.0,67334896.0,95,2024-07-11T00:00:00
103,55000000.0,68700000.0,63216300.0,62,2024-07-12T00:00:00
104,53000000.0,65000000.0,60903300.0,147,2024-07-13T00:00:00
105,60000000.0,73000000.0,67194496.0,151,2024-07-14T00:00:00


In [352]:
int(CurrentPrice['avg'].iloc[0])


72396496